# **Machine Learning Project**

In questa esercitazione metteremo assieme tutte le nozioni apprese dall'inizio del corso per risolvere un task specifico di machine learning.

### **Task: Semi-supervised classification**

Il task che vogliamo risolvere è un task di classificazione, caratterizzato però dal fatto che solo una piccola parte dei dati che disponiamo possiede le annotazioni (label). Questa condizione è nota come `Semi-supervised learning`. 

### **Dataset**

Il dataset che utilizzeremo sarà Fashion-MNIST, che contiene immagini di articoli di Zalando, composto da un training set di 60.000 campioni e test set con 10.000 campioni. Ogni campione è in scala di grigi e ha risoluzione 28x28. Il dataset è composto da 10 classi.


## **Pseudo-label**

Lo **pseudo-labeling** è una tecnica utilizzata nell'ambito del *semi-supervised learning*. L'idea di base è quella di generare etichette "artificiali" (pseudo-etichette) per i dati non etichettati (unlabeled, "U"), in modo da utilizzarle durante il training del modello. Per generare queste etichette ci sono diverse strategie: nel contesto di questa esercitazione utilizzeremo un algoritmo di clustering (k-means).

Ecco i passaggi generali del processo di pseudo-labeling:

1.  **Addestramento Iniziale**: Si addestra un algortimo di clustering sul set non etichettato (U) utilizzando un numero di cluster pari al numero di classi.
2.  **Predizione su Dati Etichettati**: Utilizziamo l'algortimo addestrato al punto 1 per clusterizzare i dati etichettati (L), assegnandoli quindi ai cluster che abbiamo trovato durante l' addestramento iniziale.
3.  **Mappare i cluster alle etichette**: Creiamo un mapping tra i cluster e le etichette, in modo da capire quale etichetta corrisponde allo specifico cluster. Per fare ciò assegniamo ad ogni cluster la vera label più frequente assegnata a quel cluster, sfruttando la funzione `mode`:

```Python
from scipy.stats import mode
import numpy as np

etichette_nel_cluster = np.array([0, 1, 1, 2, 1, 0, 1])

risultato_mode = mode(etichette_nel_cluster)

print(f"Oggetto ModeResult: {risultato_mode}") # Output: ModeResul(mode=1, count=4)
print(f"Etichetta più frequente (moda): {risultato_mode.mode}") # Output: (moda): 1

# etichetta più frequente come singolo numero:
etichetta_predominante = risultato_mode.mode
print(f"Etichetta predominante per questo cluster: {etichetta_predominante}") # Output: 1
```

4.  **Estrazione pseudo-label**: Alla fine, la classe più presente in un cluster diventa l' etichetta scelta per tutti i campioni assegnati a quel cluster. 


**Vantaggi**:
*   Permette di sfruttare la grande quantità di dati non etichettati, che altrimenti andrebbero sprecati.
*   Può migliorare significativamente le prestazioni del modello rispetto all'addestramento con i soli dati etichettati, specialmente quando questi ultimi sono scarsi.

In [225]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import mode # For majority voting
from sklearn.neural_network import MLPClassifier

In [226]:
# Nomi delle classi per Fashion-MNIST
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

### `load_and_preprocess_data()`

In questa funzione dovrete:

* Scaricare il dataset.
* Riordinare casualmente i dati.
* Effettuare reshape.
* Scalare i valori dei pixel all' intervallo [0,1].
* Ridurre il numero di campioni a 10.000 per il train e 1.000 per il test.

La funzione dovrà ritornare nel seguente ordine:

1. Il training set ridotto.
2. Le etichette di train ridotte.
3. Il test set ridotto.
4. Le etichette di test ridotte.

In [227]:
def load_and_preprocess_data():
    """Carica e pre-processa il dataset Fashion-MNIST."""
    np.random.seed(0)
    
    (x_train, y_train), (x_test, y_test)=fashion_mnist.load_data()
    indici = np.arange(x_train.shape[0])


    np.random.shuffle(indici)
    x_train=x_train[indici]
    y_train=y_train[indici]

    #x_test=x_test[indici]
    #y_test=y_test[indici]

    x_train=x_train.reshape(x_train.shape[0],-1)
    x_test=x_test.reshape(x_test.shape[0],-1)

    x_train=x_train.astype('float32')/255.0
    x_test=x_test.astype('float32')/255.0

    x_train_r=x_train[:8000]
    x_test_r=x_test[:1000]

    y_train=y_train[:8000]
    y_test=y_test[:1000]
    print("dimensioni", x_train_r.shape, y_train.shape, y_test.shape, x_test_r.shape)    
    
    return x_train_r, y_train,x_test_r,y_test


### `apply_pca_and_scale`

In questa funzione dovrete:

* Scalare il training set e il test set.
* Applicare PCA con un numero di componenti specificato come parametro della funzione (o, equivalentemente, con una frazione desiderata della varianza espressa).
* Stampare il numero di componenti.
* Stampare la varianza espressa.

La funzione dovrà ritornare nel seguente ordine:

1. Il training set trasformato con PCA.
2. Il test set trasformato con PCA.

In [228]:
def apply_pca_and_scale(x_train, x_test, n_components):
    """Applica StandardScaler e PCA."""
    scaler = StandardScaler()
    x_train_s=scaler.fit_transform(x_train)
    x_test_s=scaler.transform(x_test)

    pca = PCA(n_components=n_components) 
    x_train_pca = pca.fit_transform(x_train_s) 
    x_test_pca=pca.transform(x_test_s)
    print("componenti", pca.n_components)
    print("varianza", np.sum(pca.explained_variance_ratio_))


    return x_train_pca, x_test_pca


### `create_semi_supervised_split`

In questa funzione dovrete:

* Splittare il training set in due insiemi, etichettato (L) e non etichettato (U) utilizzando  `train_test_split` con:

`test_size`=`(1.0 - labeled_fraction)`

* Stampare la shape del set etichettato.
* Stampare la shape del set non etichettato.

La funzione deve ritornare nell seguente ordine:

1. Il set etichettato.
2. Le etichette del set etichettato.
3. Il set non etichettato.
4. Le etichette del set non etichettato. **N.B.** Queste etichette verranno utilizzate **SOLO** per valutare le pseudo-labels, non per l'addestramento.


In [229]:

def create_semi_supervised_split(x_train_pca, y_train, labeled_fraction):
    """Crea gli insiemi etichettato (L) e non etichettato (U)."""

    # Split con stratificazione per mantenere distribuzione classi
    x_label, x_unlabel, y_label, y_unlabel = train_test_split(x_train_pca, y_train,test_size=(1.0 - labeled_fraction),stratify=y_train,random_state=0)

    print("Set etichettato shape:", x_label.shape)
    print("Set non etichettato shape:", x_unlabel.shape)

    return x_label, y_label, x_unlabel, y_unlabel



### `get_pseudo_labels`

In questa funzione dovrete:

* Istanziare un algoritmo di clustering (ad esempio, k-means).
* Addestrare e predire i clustering sul set non etichettato.
* Predire i clustering del set etichettato.
* Mappare i cluster ad un etichetta, utilizzando per ogni cluster l'etichetta più presente, estraibile utilizzando la funzione `mode` presentata sopra.
* Generare un array `pseudo_labels` assegnando a ogni campione del set non etichettato l'etichetta corrispondente al cluster a cui è stato assegnato.

La funzione deve ritornare:

1. L' array `pseudo_labels`.

In [230]:
def get_pseudo_labels(x_unlabeled_pca, x_labeled_pca, y_labeled, n_clusters):

    # 1. Istanziare algoritmo di clustering
    kmeans=KMeans(n_clusters=n_clusters,random_state=42)
    
    # 2. Allenare il modello di clustering con i dati non etichettati e salvare le assegnazioni ai cluster
    kmeans.fit(x_unlabeled_pca)
    cluster_assignments_unlabeled =kmeans.fit_predict(x_unlabeled_pca)

    # 3. Calcolare l'assegnamento dei dati etichettati ai cluster
    cluster_assignments_labeled =kmeans.predict(x_labeled_pca)

    # 4. Mappiamo i cluster alle label vere più frequenti
    cluster_to_true_label_map = {}
    for k_idx in range(n_clusters):
        # 4.1 Troviamo per ogni cluster le etichette vere dei campioni che vi appartengono.
        labels_in_cluster = y_labeled[cluster_assignments_labeled == k_idx]
        # 4.2 Troviamo l'etichetta più frequente per quel cluster.
        if len(labels_in_cluster) > 0:
            most_common_l=mode(labels_in_cluster)
            most_common_l = mode(labels_in_cluster).mode
    
            # 4.3 Salviamo l'etichetta più frequente per quel cluster in cluster_to_true_label_map. 
            cluster_to_true_label_map[k_idx]=most_common_l
        else:
            cluster_to_true_label_map[k_idx]=np.nan
        # In questo dizionario, la chiave è il cluster e il valore è l'etichetta vera più frequente.

    pseudo_labels_list = []

    for c_assign in cluster_assignments_unlabeled:
        if c_assign in cluster_to_true_label_map:
            label = cluster_to_true_label_map[c_assign]
            if not np.isnan(label):
                pseudo_labels_list.append(int(label))
            else:
                pseudo_labels_list.append(np.nan)
        else:
            pseudo_labels_list.append(np.nan)
            
    
    # 5. Convertiamo la lista in un array numpy 
    pseudo_labels_array = np.array(pseudo_labels_list, dtype=np.float32)

    print(pseudo_labels_array.shape)

    return pseudo_labels_array


### `train_and_evaluate_classifier`

In questa funzione dovrete:

* Rimuovere eventuali campioni con etichette NaN (potrebbero provenire da pseudo labels non mappate).
* Istanziare il modello utilizzando `model_class` come oggetto e `classifier_args` come argomenti. Esempio:

```Python
model_class = MLPClassifier
classifier_args = {'max_iter': 200, 'hidden_layer_sizes': (100, 50)}
model = model_class(**classifier_args)

# Equivalente a:
model = MLPClassifier(max_iter=200, hidden_layer_sizes=(100, 50))

```

* Allenare il modello sul training set a cui sono stati rimossi i campioni con etichette NaN.
* Calcolare l' accuracy.
* Stampare il `title`, che consiste nel titolo dell' esperimento eseguito. Questo perchè tale funzione verrà riutilizzata diverse volte per più set di dati. Un titolo ci permetterà di identificare quali risultati stiamo producendo.
* Stampare l' accuracy.
* Stampare il classification report.

La funzione deve ritornare:

1. Il modello.


In [231]:
def train_and_evaluate_classifier(model_class, classifier_args, x_train, y_train, x_test, y_test, title, class_names_list):
    valid_indices_train = ~np.isnan(y_train)
    x_train_c = x_train[valid_indices_train]
    y_train_c = y_train[valid_indices_train]
    
    model = model_class(**classifier_args)
    model.fit(x_train_c, y_train_c) 

    y_pred=model.predict(x_test)
    
    print(title)
    accuracy=accuracy_score(y_test,y_pred)
    print("accuracy: ", accuracy)
    print("Classification Report:\n", classification_report(y_test, y_pred, target_names=class_names_list, zero_division=0))
    
    return model


### `main`

In questa funzione dovrete:

* Utilizzare la funzione `load_and_preprocess_data` per caricare e pre-processare i dati.
* Utilizzare la funzione `apply_pca_and_scale` per applicare scaling e PCA.
* Utilizzare la funzione `create_semi_supervised_split` per dividere il train set in set etichettato e non etichettato.
* Utilizzare la funzione `get_pseudo_labels` per calcoalre le pseudo labels sul set non etichettato.
* Calcolare l' accuracy delle pseudo labels, cioè confrontarle con quelle vere in modo da vedere quanto sono accurate.
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello solo sui dati etichettati (L).
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello solo sui dati non etichettati (U).
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello sui dati etichettati (L) più quelli non etichettati (U).
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello su tutto il dataset originale.

In [232]:
def main(classifier_class, classifier_args, n_components_pca, labeled_fraction, n_clusters):
    x_train_r, y_train,x_test_r,y_test=load_and_preprocess_data()
    
    x_train_pca,x_test_pca=apply_pca_and_scale(x_train_r, x_test_r, n_clusters)
    x_label, y_label, x_unlabel, y_unlabel=create_semi_supervised_split(x_train_pca, y_train, labeled_fraction)
    
    pseudo_labels_array=get_pseudo_labels(x_unlabel, x_label, y_label,n_clusters)
    valid_pseudo_indices = ~np.isnan(pseudo_labels_array)
    
    if np.any(valid_pseudo_indices):
        accuracy_pseudo = accuracy_score(y_unlabel[valid_pseudo_indices], pseudo_labels_array[valid_pseudo_indices])
        print(f"Accuracy pseudo-labels: {accuracy_pseudo:.4f}")
    else:
        print("Nessuna pseudo-label valida, accuracy non calcolata.")

    train_and_evaluate_classifier(classifier_class,classifier_args,x_label,y_label,x_test_pca,y_test,"solo dati etichettati", class_names)

    x_unlab_valid = x_unlabel[valid_pseudo_indices]
    p_label_valid = pseudo_labels_array[valid_pseudo_indices]
    train_and_evaluate_classifier(classifier_class, classifier_args, x_unlab_valid, p_label_valid, x_test_pca, y_test, "solo dati pseudo-etichettati", class_names)
    
    if len(x_label) > 0 and len(x_unlab_valid) > 0:
        x_combined = np.vstack((x_label, x_unlab_valid))
        y_combined = np.concatenate((y_label, p_label_valid))

        train_and_evaluate_classifier(classifier_class, classifier_args, x_combined, y_combined, x_test_pca, y_test, "Combinato: L + U_pseudo", class_names)
    else:
        print("Salto Scenario 3.")

    train_and_evaluate_classifier(classifier_class, classifier_args, x_train_pca, y_train, x_test_pca, y_test,"Oracle: Tutti i dati etichettati",class_names)

    

### **Utilizzare la funzione `main`**

Specifichiamo adesso un set di parametri richiesti dalla funzione main e utilizziamola. Nello specifico la funzione main ha bisogno di:

* `classifier_class`: quale classificatore utilizzare, ad esempio `'MLPClassifier'`, `'LogisticRegression'` o altri visti in precedenza.
* `classifier_args`: un dizionario contenente i parametri del classificatore scelto, ad esempio un `MLPClassifier` necessiterà del parametro `hidden_layer_sizes`. Dipendentemente da quale classificatore scegliete dovrete creare il dizionario.
* `n_components_pca`: numero di componenti di PCA che vogliamo utilizzare. Se specifichiamo un valore compreso in [0, 1] questo verrà considerato come la percentuale di varianza che vogliamo mentenere.
* `labeled_fraction`: percentuale di dati da usare come insieme etichettato. Si consiglia il valore 0.002 corrispondente allo 0.2%, cioè 16 immagini su 8000.
* `n_clusters`: numero di cluster da utilizzare, nel nostro caso vogliamo che ci sia un cluster per ogni classe, quindi 10.

Infine utilizziamo la funzione main.

In [233]:
# Parametri
CLASSIFIER_CLASS = MLPClassifier  # Modello da usare, ad esempio LogisticRegression o SVC
CLASSIFIER_ARGS = {
    'max_iter': 20,
    'hidden_layer_sizes': (200,200)  # Aumenta il numero di iterazioni per la convergenza
}
N_COMPONENTS_PCA = 0.95  # Mantiene il 95% della varianza spiegata, o un numero fisso es. 50
LABELED_FRACTION = 0.002   # Frazione di dati da usare come insieme etichettato L
N_CLUSTERS = 10          # Fashion-MNIST ha 10 classi

In [234]:
main(
    CLASSIFIER_CLASS,
    CLASSIFIER_ARGS,
    N_COMPONENTS_PCA,
    LABELED_FRACTION,
    N_CLUSTERS
)

dimensioni (8000, 784) (8000,) (1000,) (1000, 784)
componenti 10
varianza 0.6218255
Set etichettato shape: (16, 10)
Set non etichettato shape: (7984, 10)


/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


(7984,)
Accuracy pseudo-labels: 0.4792
solo dati etichettati
accuracy:  0.463
Classification Report:
               precision    recall  f1-score   support

 T-shirt/top       0.39      0.25      0.31       107
     Trouser       0.97      0.74      0.84       105
    Pullover       0.35      0.45      0.39       111
       Dress       0.42      0.77      0.54        93
        Coat       0.15      0.16      0.15       115
      Sandal       0.82      0.46      0.59        87
       Shirt       0.26      0.20      0.22        97
     Sneaker       0.73      0.73      0.73        95
         Bag       0.36      0.04      0.08        95
  Ankle boot       0.48      0.91      0.63        95

    accuracy                           0.46      1000
   macro avg       0.49      0.47      0.45      1000
weighted avg       0.48      0.46      0.44      1000



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


solo dati pseudo-etichettati
accuracy:  0.434
Classification Report:
               precision    recall  f1-score   support

 T-shirt/top       0.17      0.25      0.20       107
     Trouser       0.61      0.92      0.73       105
    Pullover       0.36      0.63      0.46       111
       Dress       0.16      0.23      0.19        93
        Coat       0.00      0.00      0.00       115
      Sandal       0.52      0.64      0.58        87
       Shirt       0.00      0.00      0.00        97
     Sneaker       0.64      0.79      0.71        95
         Bag       0.00      0.00      0.00        95
  Ankle boot       0.66      0.93      0.77        95

    accuracy                           0.43      1000
   macro avg       0.31      0.44      0.36      1000
weighted avg       0.31      0.43      0.36      1000



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Combinato: L + U_pseudo
accuracy:  0.431
Classification Report:
               precision    recall  f1-score   support

 T-shirt/top       0.16      0.23      0.19       107
     Trouser       0.61      0.92      0.73       105
    Pullover       0.36      0.63      0.46       111
       Dress       0.16      0.22      0.18        93
        Coat       0.00      0.00      0.00       115
      Sandal       0.48      0.63      0.54        87
       Shirt       0.00      0.00      0.00        97
     Sneaker       0.66      0.80      0.72        95
         Bag       0.00      0.00      0.00        95
  Ankle boot       0.66      0.93      0.77        95

    accuracy                           0.43      1000
   macro avg       0.31      0.44      0.36      1000
weighted avg       0.30      0.43      0.35      1000

Oracle: Tutti i dati etichettati
accuracy:  0.814
Classification Report:
               precision    recall  f1-score   support

 T-shirt/top       0.84      0.77      0.80    

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
